In [115]:
import vcfpy
import pandas as pd
import numpy as np
import gzip

In [78]:
chr22 = vcfpy.Reader.from_path('../raw_data/ALL.chr22.phase3_shapeit2_mvncall_integrated_v5a.20130502.genotypes.vcf.gz')
writer = vcfpy.Writer.from_path('../raw_data/ALL.chr22.phase3.10k.vcf', chr22.header)
for record in chr22.fetch('22', 29130000, 29140000):
    writer.write_record(record)
    

In [141]:
chr22 = vcfpy.Reader.from_path('../raw_data/ALL.chr22.phase3.10k.vcf.gz')
header = ['#CHROM', 'POS', 'ID', 'REF', 'ALT'] + chr22.header.samples.names
#print('\t'.join(header))
line = []
ar = []
for record in chr22.fetch('22', 29137700, 29137800):
    line = [record.CHROM, record.POS, record.ID[0], record.REF]
    line += [alt.value for alt in record.ALT[:1]]
    #print(list(record.ALT)[0].value)
    line += [call.data.get('GT') or './.' for call in record.calls]
    #print('\t'.join(map(str, line))
    ar.append(line)
#print(ar)
dfSNP = pd.DataFrame(ar)
dfSNP.columns = header
dfSNP

,#CHROM,POS,ID,REF,ALT,HG00096,HG00097,HG00099,HG00100,HG00101,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,22,29137730,rs74501869,C,G,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
1,22,29137736,rs185897360,G,T,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,22,29137758,rs539227672,G,A,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
3,22,29137761,rs554107994,T,G,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,22,29137770,rs566344661,C,G,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
5,22,29137782,rs536566373,A,G,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
6,22,29137790,rs142541707,C,A,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [205]:
for idx, row in dfSNP.iloc[:, :2].head(2).iterrows():
    if idx ==0:
        row['POS'] = ''
    print(row)

#CHROM    22
POS         
Name: 0, dtype: object
#CHROM          22
POS       29130025
Name: 1, dtype: int64


In [185]:
with gzip.open('../raw_data/ALL.chr22.SNPs.10k.vcf.gz', 'rb') as handle:
    dfSNP = pd.read_csv(handle, sep='\t')
    dfTmp = dfSNP[(dfSNP['POS']>29130001) & (dfSNP['POS']<29140001)]
    dfTmp  = dfTmp[dfTmp['ALT'].apply(lambda x: len(x)==1)].reset_index(drop=True)
dfTmp.head()


,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO,FORMAT,HG00096,...,NA21128,NA21129,NA21130,NA21133,NA21135,NA21137,NA21141,NA21142,NA21143,NA21144
0,22,29130012,rs738722,T,C,100,PASS,AC=3499;AF=0.698682;AN=5008;NS=2504;DP=20192;E...,GT,0|0,...,0|1,1|1,1|1,1|1,1|1,0|1,1|1,1|1,1|1,1|1
1,22,29130025,rs578074334,G,C,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=19624;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,22,29130038,rs17880388,C,T,100,PASS,AC=29;AF=0.00579073;AN=5008;NS=2504;DP=18622;E...,GT,0|0,...,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|1,1|0,0|0
3,22,29130044,rs554442856,C,G,100,PASS,AC=1;AF=0.000199681;AN=5008;NS=2504;DP=18125;E...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,22,29130057,rs190226295,G,A,100,PASS,AC=8;AF=0.00159744;AN=5008;NS=2504;DP=17450;EA...,GT,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [170]:
k = ['NA18488', 'NA19213', 'NA19209']
dfTmp = dfSNP[k]

for idx, row in dfSNP.iterrows():
    print(dfTmp.columns[row[[k]] != '0|0'])

KeyError: "None of [[['NA18488', 'NA19213', 'NA19209']]] are in the [index]"

In [167]:
("[1,2,3]").strip('[').strip(']').split(',')

['1', '2', '3']

In [163]:
dfSNP['ALT'].str.len() ==1

0    True
1    True
2    True
3    True
4    True
5    True
6    True
Name: ALT, dtype: bool

In [122]:
sel

['HG01390',
 'HG01889',
 'HG02455',
 'HG02943',
 'HG02982',
 'HG03084',
 'HG03291',
 'HG03514',
 'HG03538',
 'HG03687',
 'HG03950',
 'NA11918',
 'NA19209',
 'NA19213',
 'NA19320',
 'NA19380',
 'NA19454',
 'NA19455',
 'NA19717']

In [171]:
['NA18488', 'NA19213', 'NA19209'] + [1]

['NA18488', 'NA19213', 'NA19209', 1]

In [ ]:
'ACCTGGAGCCGCACACTCTCCGC'


In [197]:
cols = ['HG01390', 'HG01889', 'HG02455', 'HG02943','HG02982', 'HG03084', 'HG03291', 'HG03514', 'HG03538', 'HG03687', 'HG03950','NA11918', 'NA19209', 'NA19213', 'NA19320', 'NA19380', 'NA19454', 'NA19455','NA19717']
dfSample[dfSample['Individual ID'].isin(cols)]

,Family ID,Individual ID,Paternal ID,Maternal ID,Gender,Phenotype,Population,Relationship,Siblings,Second Order,Third Order,Children,Other Comments,phase 3 genotypes,related genotypes,omni genotypes,affy_genotypes
616,CLM42,HG01390,0,0,2,0,CLM,mother,0,0,0,HG01391,0,1,0,1,1
922,BB05,HG01889,0,0,2,0,ACB,mother,0,0,0,HG01891,0,1,0,1,1
1289,BB48,HG02455,0,0,1,0,ACB,father,0,0,0,0,0,1,0,0,1
1563,NG13,HG02943,0,0,2,0,ESN,mother,0,0,0,HG02945,0,1,0,0,1
1589,GB97,HG02982,0,0,1,0,GWD,father,0,0,0,HG02984,0,1,0,0,1
1649,HG03084,HG03084,0,0,1,0,MSL,unrels,0,0,0,0,0,1,0,0,1
1763,NG72,HG03291,0,0,2,0,ESN,mother,0,0,0,HG03293,0,1,0,0,1
1885,NG107,HG03514,0,0,2,0,ESN,mother,0,0,0,HG03516,0,1,0,0,1
1894,GB125,HG03538,0,0,1,0,GWD,father,0,0,0,HG03540,0,1,0,0,1
1987,ST015,HG03687,0,0,1,0,STU,father,0,0,0,0,0,1,0,0,1


In [199]:
cols = ['#CHROM', 'POS', 'ID', 'REF', 'ALT'] + ['HG01390', 'HG01889', 'HG02455', 'HG02943','HG02982', 'HG03084', 'HG03291', 'HG03514', 'HG03538', 'HG03687', 'HG03950','NA11918', 'NA19209', 'NA19213', 'NA19320', 'NA19380', 'NA19454', 'NA19455','NA19717']

In [203]:
dfSNP[(dfSNP['POS'] > 29137769) & (dfSNP['POS']< 29137795)][cols]

,#CHROM,POS,ID,REF,ALT,HG01390,HG01889,HG02455,HG02943,HG02982,...,HG03687,HG03950,NA11918,NA19209,NA19213,NA19320,NA19380,NA19454,NA19455,NA19717
215,22,29137770,rs566344661,C,G,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|1
216,22,29137782,rs536566373,A,G,0|0,0|0,0|0,0|0,0|0,...,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
217,22,29137790,rs142541707,C,A,0|1,1|0,1|0,1|0,1|0,...,0|0,0|0,0|0,0|1,0|1,1|0,0|0,0|0,0|1,0|0


In [121]:
dfSNP[list(set(yri))]

,NA18870,NA18865,NA18907,NA18923,NA18934,NA19108,NA18511,NA18488,NA19144,NA18486,...,NA19114,NA18912,NA19117,NA19098,NA19118,NA19143,NA19171,NA18519,NA18881,NA19236
0,0|0,0|0,0|0,1|1,1|0,0|0,0|0,0|0,1|0,0|0,...,0|0,0|0,0|0,0|1,0|0,0|0,0|0,0|0,0|0,0|0
1,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
2,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
3,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
4,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
5,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0
6,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,...,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0,0|0


In [72]:
dfSNP[dfSNP[2508].notnull()]

,0,1,2,3,4,5,6,7,8,9,...,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508
31,22,29130813,G,A,GA,2|2,0|2,0|2,2|0,0|0,...,1|0,0|0,0|1,1|0,0|1,2|0,0|1,0|1,0|0,0|0
83,22,29132752,AC,ACC,A,2|2,2|2,0|2,2|0,0|0,...,0|0,0|0,0|0,0|0,0|2,2|0,0|0,0|0,0|0,0|0


In [66]:
dfSNP[list(set(yri) & set(sel))]

,NA19213,NA19209
0,0|0,0|0
1,0|0,0|0
2,0|0,0|0
3,0|0,0|0
4,0|1,0|1


In [159]:
dfTmp = dfSNP[sel]
for idx, row in dfTmp.iterrows():
    if len(dfTmp.columns[(row != '0|0')])>0:
        print("with changes")
    else:
        print("No dividing groups")

No dividing groups
No dividing groups
with changes
with changes
with changes
with changes
with changes


In [56]:
sel = []
cols = chr22.header.samples.names
for c in cols:
    if not (dfSNP[c] == '0|0').all():
        sel.append(c)
sel

['HG01390',
 'HG01889',
 'HG02455',
 'HG02943',
 'HG02982',
 'HG03084',
 'HG03291',
 'HG03514',
 'HG03538',
 'HG03687',
 'HG03950',
 'NA11918',
 'NA19209',
 'NA19213',
 'NA19320',
 'NA19380',
 'NA19454',
 'NA19455',
 'NA19717']

In [6]:
dfSample = pd.read_csv('../raw_data/sample_individuals_ALL_v2.20130502.tsv', sep='\t')
dfSample = dfSample[dfSample['phase 3 genotypes']==1]
dfSample.shape

(2504, 17)

In [7]:
dfSample.head()

,Family ID,Individual ID,Paternal ID,Maternal ID,Gender,Phenotype,Population,Relationship,Siblings,Second Order,Third Order,Children,Other Comments,phase 3 genotypes,related genotypes,omni genotypes,affy_genotypes
0,HG00096,HG00096,0,0,1,0,GBR,unrel,0,0,0,0,0,1,0,1,1
1,HG00097,HG00097,0,0,2,0,GBR,unrel,0,0,0,0,0,1,0,1,1
3,HG00099,HG00099,0,0,2,0,GBR,unrel,0,0,0,0,0,1,0,1,1
4,HG00100,HG00100,0,0,2,0,GBR,unrel,0,0,0,0,0,1,0,1,1
5,HG00101,HG00101,0,0,1,0,GBR,unrel,0,0,0,0,0,1,0,1,1


In [49]:
dfSample[(dfSample['Gender']==1) & (dfSample['Relationship'].str.contains('unrel')) & (dfSample['Population'].str.contains('CEU'))]

,Family ID,Individual ID,Paternal ID,Maternal ID,Gender,Phenotype,Population,Relationship,Siblings,Second Order,Third Order,Children,Other Comments,phase 3 genotypes,related genotypes,omni genotypes,affy_genotypes


In [59]:
dfSample[dfSample['Individual ID'].isin(sel)]

,Family ID,Individual ID,Paternal ID,Maternal ID,Gender,Phenotype,Population,Relationship,Siblings,Second Order,Third Order,Children,Other Comments,phase 3 genotypes,related genotypes,omni genotypes,affy_genotypes
616,CLM42,HG01390,0,0,2,0,CLM,mother,0,0,0,HG01391,0,1,0,1,1
922,BB05,HG01889,0,0,2,0,ACB,mother,0,0,0,HG01891,0,1,0,1,1
1289,BB48,HG02455,0,0,1,0,ACB,father,0,0,0,0,0,1,0,0,1
1563,NG13,HG02943,0,0,2,0,ESN,mother,0,0,0,HG02945,0,1,0,0,1
1589,GB97,HG02982,0,0,1,0,GWD,father,0,0,0,HG02984,0,1,0,0,1
1649,HG03084,HG03084,0,0,1,0,MSL,unrels,0,0,0,0,0,1,0,0,1
1763,NG72,HG03291,0,0,2,0,ESN,mother,0,0,0,HG03293,0,1,0,0,1
1885,NG107,HG03514,0,0,2,0,ESN,mother,0,0,0,HG03516,0,1,0,0,1
1894,GB125,HG03538,0,0,1,0,GWD,father,0,0,0,HG03540,0,1,0,0,1
1987,ST015,HG03687,0,0,1,0,STU,father,0,0,0,0,0,1,0,0,1


In [62]:
yri = list(dfSample[dfSample['Population'].str.contains('YRI')]['Individual ID'].values)
yri

['NA18486',
 'NA18488',
 'NA18489',
 'NA18498',
 'NA18499',
 'NA18501',
 'NA18502',
 'NA18504',
 'NA18505',
 'NA18507',
 'NA18508',
 'NA18510',
 'NA18511',
 'NA18516',
 'NA18517',
 'NA18519',
 'NA18520',
 'NA18522',
 'NA18523',
 'NA18853',
 'NA18856',
 'NA18858',
 'NA18861',
 'NA18864',
 'NA18865',
 'NA18867',
 'NA18868',
 'NA18870',
 'NA18871',
 'NA18873',
 'NA18874',
 'NA18876',
 'NA18877',
 'NA18878',
 'NA18879',
 'NA18881',
 'NA18907',
 'NA18908',
 'NA18909',
 'NA18910',
 'NA18912',
 'NA18915',
 'NA18916',
 'NA18917',
 'NA18923',
 'NA18924',
 'NA18933',
 'NA18934',
 'NA19092',
 'NA19093',
 'NA19095',
 'NA19096',
 'NA19098',
 'NA19099',
 'NA19102',
 'NA19107',
 'NA19108',
 'NA19113',
 'NA19114',
 'NA19116',
 'NA19117',
 'NA19118',
 'NA19119',
 'NA19121',
 'NA19129',
 'NA19130',
 'NA19131',
 'NA19137',
 'NA19138',
 'NA19141',
 'NA19143',
 'NA19144',
 'NA19146',
 'NA19147',
 'NA19149',
 'NA19152',
 'NA19153',
 'NA19159',
 'NA19160',
 'NA19171',
 'NA19172',
 'NA19175',
 'NA19184',
 'NA

In [48]:
dfSample.shape[0]

2504

In [14]:
dfSample['Relationship'].value_counts()

unrel                      917
mother                     731
father                     720
mat grandmother             24
child                       23
mat grandfather             23
pat grandfather             22
pat grandmother             22
unrels                      13
not father                   3
Child                        1
pat grandfather; father      1
pat grandmother; mother      1
husband of Child             1
wife of child                1
mother; child                1
Name: Relationship, dtype: int64

In [108]:
from dataclasses import asdict
import numpy as np
import bwt
import imp
imp.reload(bwt)

<module 'bwt' from '/home/james/Documents/BMES550/CRISPRandME/code/bwt.py'>

In [109]:
t = bwt.BWT(T='CTAGCTAGCTAGCGGATCGTATATGCTAGCTAGCGG')
print(t)

BWT(T='CTAGCTAGCTAGCGGATCGTATATGCTAGCTAGCGG', charfreq=None, charspace=[], start=29130001, stop=29140000)


In [110]:
t.main()

SNP list complete!
BWT complete!
[[False False  True False]
 [False False False  True]
 [False False False  True]
 [False False False  True]
 [False False False  True]
 [False False False  True]
 [False False False  True]
 [False False  True False]
 [False False False  True]
 [False False  True False]
 [False False  True False]
 [False False False  True]
 [False False  True False]
 [False False  True False]
 [False False  True False]
 [False False  True False]
 [False False False False]
 [False False  True False]
 [False False  True False]
 [ True False False False]
 [ True False False False]
 [ True False False False]
 [ True False False False]
 [False False False  True]
 [ True False False False]
 [False  True False False]
 [False  True False False]
 [False  True False False]
 [False  True False False]
 [False  True False False]
 [False  True False False]
 [False  True False False]
 [False  True False False]
 [False False  True False]
 [ True False False False]
 [ True False False Fa

In [113]:
t.bwstr

'GTTTTTTGTGGTGGGG$GGAAAATACCCCCCCCGAAA'

In [13]:
t.charfreq

Counter({'C': 8, 'T': 9, 'A': 8, 'G': 11, '$': 1})

In [14]:
list(t.sufidx)

[36,
 31,
 10,
 27,
 6,
 2,
 20,
 15,
 22,
 33,
 12,
 17,
 29,
 8,
 25,
 4,
 0,
 35,
 14,
 32,
 11,
 28,
 7,
 24,
 3,
 34,
 13,
 18,
 30,
 9,
 26,
 5,
 1,
 19,
 21,
 16,
 23]

In [15]:
t.cp

array([[  0.,   0.,   1.,   0.],
       [  0.,   0.,   1.,   1.],
       [  0.,   0.,   1.,   2.],
       [  0.,   0.,   1.,   3.],
       [  0.,   0.,   1.,   4.],
       [  0.,   0.,   1.,   5.],
       [  0.,   0.,   1.,   6.],
       [  0.,   0.,   2.,   6.],
       [  0.,   0.,   2.,   7.],
       [  0.,   0.,   3.,   7.],
       [  0.,   0.,   4.,   7.],
       [  0.,   0.,   4.,   8.],
       [  0.,   0.,   5.,   8.],
       [  0.,   0.,   6.,   8.],
       [  0.,   0.,   7.,   8.],
       [  0.,   0.,   8.,   8.],
       [  0.,   0.,   8.,   8.],
       [  0.,   0.,   9.,   8.],
       [  0.,   0.,  10.,   8.],
       [  1.,   0.,  10.,   8.],
       [  2.,   0.,  10.,   8.],
       [  3.,   0.,  10.,   8.],
       [  4.,   0.,  10.,   8.],
       [  4.,   0.,  10.,   9.],
       [  5.,   0.,  10.,   9.],
       [  5.,   1.,  10.,   9.],
       [  5.,   2.,  10.,   9.],
       [  5.,   3.,  10.,   9.],
       [  5.,   4.,  10.,   9.],
       [  5.,   5.,  10.,   9.],
       [  

In [16]:
t.query('TAGCGG')

Initial first index: [17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
current checking c G
bwt 17 is G
crank is 9
nextidx = 25
bwt 18 is G
crank is 10
nextidx = 26
[25, 26]
current checking c C
bwt 25 is C
crank is 1
nextidx = 9
bwt 26 is C
crank is 2
nextidx = 10
[9, 10]
current checking c G
bwt 9 is G
crank is 3
nextidx = 19
bwt 10 is G
crank is 4
nextidx = 20
[19, 20]
current checking c A
bwt 19 is A
crank is 1
nextidx = 1
bwt 20 is A
crank is 2
nextidx = 2
[1, 2]
current checking c T
bwt 1 is T
crank is 1
nextidx = 28
bwt 2 is T
crank is 2
nextidx = 29
[28, 29]
Resolving [28, 29]
CURRENT FOUND [30]
CURRENT FOUND [30, 9]
Result:
TAGCGG [30, 9]


In [51]:
example='CCGCAGCCTCAGCCAGCAGAGTG'

In [50]:
from Bio.Seq import Seq

In [54]:
Seq(example).reverse_complement()

Seq('CACTCTGCTGGCTGAGGCTGCGG', Alphabet())

In [98]:
bw = []
a = np.array([1,0,0,0], dtype=np.bool)
b = np.array([0,0,0,1], dtype=np.bool)
bw.append(a)
bw.append(b)
bw = np.array(bw)
bw.shape

(2, 4)

In [103]:
refchar = 'G'
nuc_ar = np.array(['A', 'C', 'G', 'T'])
altchar = 'T'
(nuc_ar == refchar) | (nuc_ar == altchar)

array([False, False,  True,  True], dtype=bool)

In [106]:
chars = np.array(['A', 'G'], dtype=np.string_)

In [107]:
chars

array([b'A', b'G'], 
      dtype='|S1')